# Traffic Indicators on I-94

## Introduction

In this project, we will analyze a dataset about the westbound traffic on the I-94 Interstate highway using a [data set](https://archive.ics.uci.edu/ml/datasets/Metro+Interstate+Traffic+Volume) made available by John Hogue. I-94 runs from Billings, Montana, to Port Huron, Michigan through the Great Lakes region in the northern United States. Our data set tracks westbound traffic volume in addition to weather, temperature, date/time, and whether the day was a holiday. The data was acquired from a station located approximately midway between Minneapolis and Saint Paul.

The goal of our analysis is to determine some indicators of heavy traffic on I-94--indicators such as weather type, time of the day, time of the week, etc. with the goal of avoiding future traffic ourselves when headed west on I-94.

### Importing and Inspecting the Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_theme()
traffic = pd.read_csv('../Datasets/Metro_Interstate_Traffic_Volume.csv')

| variable | type | description |
| --- | --- | --- |
| holiday | Categorical | US National holidays plus regional holiday, Minnesota State Fair |
| temp | Numeric | Average temp in kelvin |
| rain_1h | Numeric | Amount in mm of rain that occurred in the hour |
| snow_1h | Numeric | Amount in mm of snow that occurred in the hour |
| clouds_all | Numeric | Percentage of cloud cover |
| weather_main | Categorical | Short textual description of the current weather |
| weather_description | Categorical | Longer textual description of the current weather |
| date_time | DateTime | Hour of the data collected in local CST time |
| traffic_volume | Numeric | Hourly I-94 ATR 301 reported westbound traffic volume |

Let's take a quick look at the data

In [ ]:
traffic.head()

In [ ]:
traffic.tail()

In [ ]:
traffic.info()

Let's inspect the distribution of our main variable, `traffic_volume`.

In [ ]:
plt.hist(traffic['traffic_volume'], bins=30)
plt.show()

In [ ]:
traffic['traffic_volume'].describe()

Westbound traffic on I-94 shows a trimodal distribution, indicating that it has three common states: low--roughly 250-500 cars hourly; medium--roughly 3000 cars hourly; or high--roughly 4500-5000 cars hourly. These roughly track our data's 25th, 50th, and 75th percentile.

## Analyzing the Data

One obvious factor that may regularly traffic volume is time of day. Thus, we'll first take a look at how time of day affects traffic volume.

### Effects of Day vs. Night

The first way we'll look into time effects is categorically, dividing the data into daytime and night time subsets. First, though, we'll change the type of the `date_time` variable into datetime.

In [ ]:
traffic['date_time'] = pd.to_datetime(traffic['date_time'])

Now we'll get the hour of each entry to use this in separating the data into day and night.

In [ ]:
traffic['hour'] = traffic['date_time'].dt.hour

In [ ]:
traffic['hour'].describe()

Hours run from 0 to 23. We might resonably consider daytime hours to be 7:00 am - 7:00 pm and night time hours to be 7:00 pm - 7:00 am. 

In [ ]:
daynight = {}
daynight['Day'] = traffic[(traffic['hour'] > 7) & (traffic['hour'] <= 19)]
daynight['Night'] = traffic[(traffic['hour'] <= 7) | (traffic['hour'] > 19)]

Now, let's have a look at the distribution of traffic volume for day and night.

In [ ]:
times = ['Day', 'Night']
plt.figure(figsize=(10,4))

for i, time in zip(range(1,3), times):
    plt.subplot(1,2,i)
    plt.hist(daynight[time]['traffic_volume'], bins=20)
    plt.title(time)
    plt.xlabel('Traffic Volume (cars/hr)')
    plt.ylabel('Frequency')
    plt.xlim([0,8000])
    plt.ylim([0,5000])
    
plt.show()

In [ ]:
for t in times:
    print(t)
    print(daynight[t]['traffic_volume'].describe())
    print('\n')

It's clear from these graphs that the two times of day have very different distributions. Daytime traffic seems to be unimodal with a slight negative skew. The daytime peak is around 5000 cars/hr. Night time traffic, however, seems to be trimodal, similar to the overall data set, with a strong low traffic peak, a weaker medium traffic peak, and a small high traffic peak. 

### Effects of Hour of Day

Now let's inspect the traffic by time in a more granular way by inpecting hourly traffic. First, we'll get make a plot that helps us see the distribution, then we'll plot the average hourly traffic.

In [ ]:
hourly_avgs = traffic.groupby('hour').mean()[['traffic_volume']].reset_index()

In [ ]:
plt.figure(figsize=(10,4))

# Violin Plot
plt.subplot(1,2,1)
sns.violinplot(data = traffic, x = "hour",
             y = "traffic_volume")
plt.xlabel('Hour of Day')
plt.ylabel('Traffic Volume (cars/hr)')
plt.title("Traffic Volume by Hour of Day")
plt.ylim([0,8500])

# Bar Plot of Averages
plt.subplot(1,2,2)
plt.bar(hourly_avgs['hour'], hourly_avgs['traffic_volume'])
plt.xticks(range(0,24))
plt.xlabel('Hour of Day')
plt.ylabel('Mean Traffic Volume (cars/hr)')
plt.title("Traffic Volume by Hour of Day")
plt.ylim([0,8500])

plt.show()

These plots show that from the hours of roughly 11:00 pm (23:00) to 5:00 am, traffic is relatively calm. However, between about 6:00 am and 6:00 pm, traffic is at its highest, with the hours of 6:00 - 8:00 am and 3:00 - 5:00 pm (15:00 - 17:00) being the most variable. The highest average traffic is from 3:00 - 5:00 pm, and the lowest average traffic is from 1:00 - 3:00 am.

### Effect of Month on Traffic

Next, we'll look into effects of month of the year on traffic volume.

First, we'll add a month variable to our data set.

In [ ]:
traffic['month'] = traffic['date_time'].dt.month

In [ ]:
traffic['month'].describe()

Next, we'll get the average monthly traffic and plot the distribution and the average.

In [ ]:
monthly_avgs = traffic.groupby('month').mean()[['traffic_volume']].reset_index()

In [ ]:
plt.figure(figsize=(10,4))

# Violin Plot
plt.subplot(1,2,1)
sns.violinplot(data = traffic, x = "month",
             y = "traffic_volume")
plt.xlabel('Month of Year')
plt.ylabel('Mean Traffic Volume (cars/hr)')
plt.title("Traffic Volume by Month of Year")
plt.ylim([0,8500])

# Bar Plot of Averages
plt.subplot(1,2,2)
plt.bar(monthly_avgs['month'], monthly_avgs['traffic_volume'])
plt.xticks(range(1,13))
plt.xlabel('Month of Year')
plt.ylabel('Mean Traffic Volume (cars/hr)')
plt.title("Traffic Volume by Month of Year")
plt.ylim([0,8500])

plt.show()

The violin plot is very difficult to discern, and the average differences are hard to see without zooming the plot in, so let's do that.

In [ ]:
plt.bar(monthly_avgs['month'], monthly_avgs['traffic_volume'])
plt.xlabel('Month of Year')
plt.ylabel('Mean Traffic Volume (cars/hr)')
plt.title("Traffic Volume by Month of Year")
plt.xticks(range(1,13))
plt.ylim([2900,3500])

plt.show()

These figures show that average traffic does not change dramatically across months, but is highest in June, August, and October and lowest from January to December. Average traffic ramps up slightly from winter to summer and back down as winter approaches, with the interesting exception that traffic dips in July.

### Effect of Day of the Week on Traffic

Next, we'll look at effects of weekday on traffic. We'll follow the same protocol, creating a day variable, then averaging and plotting by day.

In [ ]:
traffic['dayofweek'] = traffic['date_time'].dt.dayofweek

In [ ]:
traffic['dayofweek'].describe()

Day of week is described by the numbers 0 (Monday) to 7 (Sunday).

In [ ]:
daily_avgs = traffic.groupby('dayofweek').mean()[['traffic_volume']].reset_index()

In [ ]:
plt.figure(figsize=(10,4))

# Violin Plot
plt.subplot(1,2,1)
sns.violinplot(data = traffic, x = "dayofweek",
             y = "traffic_volume")
plt.xlabel('Day of the Week')
plt.ylabel('Mean Traffic Volume (cars/hr)')
plt.title("Traffic Volume by Day of the Week")
#plt.ylim([0,8500])

# Bar Plot of Averages
plt.subplot(1,2,2)
plt.bar(daily_avgs['dayofweek'], daily_avgs['traffic_volume'])
plt.xticks(range(0,7))
plt.xlabel('Day of the Week')
plt.ylabel('Mean Traffic Volume (cars/hr)')
plt.title("Traffic Volume by Day of the Week")
#plt.ylim([0,8500])

plt.show()

We can see by these plots that traffic is much lower on average on weekends compared to during the week, and the average slowly ramps up from Monday to Friday fairly regularly, with the overall trend looking very similar across the business week. Interestingly, the violin plots during the week show the trimodal tendency seen in the overall data, with the largest frequency being in high traffic times.

Because of these data, we may want to reconsider the hourly trends, which may be different from business days to weekends. 

### Effect of Time of Day by Business Day vs. Weekend on Traffic

In [ ]:
traffic['business_day'] = np.where(traffic['dayofweek'] <= 4, 'Business Day', 'Weekend')

In [ ]:
traffic['business_day'].value_counts()

Now, take the averages per hour, grouping  by the `business_day` variable.

In [ ]:
business_day_avgs = traffic.groupby(['business_day', 'hour']).mean()[['traffic_volume']].reset_index()

Now plot.

In [ ]:
plt.figure(figsize=(10,10))

# Violin Plots
for i, day in zip(range(1,3), traffic['business_day'].unique()):
    plt.subplot(2,2,i)
    sns.violinplot(data = traffic[traffic['business_day'] == day], x = "hour",
                 y = "traffic_volume")
    plt.xlabel('Hour of Day')
    plt.ylabel('Traffic Volume (cars/hr)')
    plt.title(day)
    plt.ylim([0,8500])

# Bar Plots of Averages
for i, day in zip(range(3,5), traffic['business_day'].unique()):
    plt.subplot(2,2,i)
    plt.bar(business_day_avgs.loc[business_day_avgs['business_day']==day,'hour'], business_day_avgs.loc[business_day_avgs['business_day']==day,'traffic_volume'])
    plt.xticks(range(0,24))
    plt.xlabel('Hour of Day')
    plt.ylabel('Mean Traffic Volume (cars/hr)')
    plt.title(day)
    plt.ylim([0,8500])

plt.show()


These plots make clear that the weekday traffic trend is more strongly bimodal, with traffic peaking in the pre-work rush hour around 7:00 am and in the post-work rush hour around 4:00 pm (16:00). Traffic stays relatively high during weekdays bewteen 6:00 am and 6:00 pm. 

On weekends, the distribution is strongly unimodal, looking roughly the same as weekdays if the pre- and post-work peaks were shaved down and the distribution were moved roughly three hours later. Traffic on weekeds has a lower peak, but is strongest between 8:00 am and 10:00 pm.

### Influence of Weather on Traffic

We have already seen that winter traffic is lower than summer traffic, suggesting heat and weather may have a strong influence. Given that we have weather variables to inspect, we can now take a look at those.

Here, we can take a more correlational approach, looking directly at the relationship between traffic volume and our weather variables, `temp`, `rain_1h`, `snow_1h`, and `clouds_all`. We also have descriptive variables, `weather_main` and `weather_description`, which are non-numeric. We might treat these categorically.

First, let's look at a description of these variables.

In [ ]:
print(traffic[['temp', 'rain_1h', 'snow_1h', 'clouds_all', 'weather_main', 'weather_description']].head())

In [ ]:
traffic[['temp', 'rain_1h', 'snow_1h', 'clouds_all', 'weather_main', 'weather_description']].describe()

#### Numerical Weather Variables

Now, let's look at the correlation of traffic volume with the numerical weather variables.

Temperature has some points listed as 0 Kelvin, which is impossible, so we'll omit those first.

In [ ]:
traffic = traffic[traffic['temp'] > 0]
traffic[['traffic_volume','temp', 'rain_1h', 'snow_1h', 'clouds_all']].corr()['traffic_volume']

In [ ]:
from scipy.stats import pearsonr
print(pearsonr(traffic['temp'], traffic['traffic_volume']))
print(pearsonr(traffic['rain_1h'], traffic['traffic_volume']))
print(pearsonr(traffic['snow_1h'], traffic['traffic_volume']))
print(pearsonr(traffic['clouds_all'], traffic['traffic_volume']))


It looks like there are significant correlations of traffic volume with both temperature (r = .13) and percentage of cloud cover (r = .07). Both are positive, indicating that traffic volume increases with increased temperature, and paradoxically, increased cloud cover. 

Let's see what these correlations look like when plotted.

In [ ]:
traffic[traffic['temp'] > 0].plot.scatter(x='temp',
                     y='traffic_volume', alpha=.1)
plt.xlabel('Temperature (K)')
plt.ylabel('Traffic Volume (cars/hr)')
plt.title('Relationship of Temparature and Traffic Volume')
plt.show()

In [ ]:
traffic.plot.scatter(x='clouds_all',
                     y='traffic_volume', alpha=.1)
plt.xlabel('% Cloud Cover')
plt.ylabel('Traffic Volume (cars/hr)')
plt.title('Relationship of Cloud Cover and Traffic Volume')
plt.show()

Neither Temperature nor cloud cover have a strong predictive value for traffic volume, as is clear from their low correlation values and scatter plots showing no obvious relationship between the variables. While statistical tests show a very small relationship between these variables and traffic volume, they account for very little variance in traffic and would not individually be good indicators of the predicted traffic volume.

#### Categorical Weather Variables

Now, let's look at the categorical weather variables.

We'll first take a look at the unique values and their counts for both variables. Weather description seems to have some duplicates due to capitalization, so we'll make all entries lowercase.

In [ ]:
w_main_counts = traffic['weather_main'].value_counts()
print(w_main_counts)
w_main_unique = traffic['weather_main'].unique()

In [ ]:
traffic['weather_description'].str.lower()

In [ ]:
w_desc_counts = traffic['weather_description'].str.lower().value_counts()
print(w_desc_counts)

We're going to get the average traffic volume for each category and compare them, but we don't want to take the average traffic across labels with very few points, because those averages will be less reliable indicators of what the traffic volume is likely to be. Therefore, we'll set a cutoff at greater than or equal to twenty entries with a given label and average across those for each category.

First, let's subset the main dataframe for each variable with greater than 20 entries.

In [ ]:
traffic_w_main_20 = traffic[traffic['weather_main'].isin(w_main_counts[w_main_counts >= 20].index)]
traffic_w_desc_20 = traffic[traffic['weather_description'].isin(w_desc_counts[w_desc_counts >= 20].index)]

Now, let's get the average for each label in these two variables for their respective data and plot them.

In [ ]:
w_main_means = traffic_w_main_20.groupby('weather_main').mean()[['traffic_volume']].reset_index().sort_values('traffic_volume')
w_desc_means = traffic_w_desc_20.groupby('weather_description').mean()[['traffic_volume']].reset_index().sort_values('traffic_volume')

In [ ]:
plt.barh(y=w_main_means['weather_main'], width=w_main_means['traffic_volume'] )
plt.xlim(2500,3700)
plt.title('Traffic Volume by Weather')
plt.xlabel('Traffic Volume (cars/hr)')
plt.ylabel('Weather')
plt.show()

This plot shows that cloudy and hazy days increase traffic volume the most compared to clear weather, though rain, drizzle and smoke also cause some increase. On the other hand, fog decreases traffic volume the most, though snow, thunderstorm, and mist do as well.

In [ ]:
plt.figure(figsize=(10,10))
plt.barh(y=w_desc_means['weather_description'], width=w_desc_means['traffic_volume'] )
plt.xlim(2500,4700)
plt.title('Traffic Volume by Weather (decription)')
plt.xlabel('Traffic Volume (cars/hr)')
plt.ylabel('Weather')
plt.show()

Looking at the more granular descriptions, it is clear that proximity shower rains increase traffic volume most strongly, adding roughly 750 more cars per hour compared to the next worst weather description, scattered clouds. Thunderstorms with heavy rain cause less traffic than any other weather description.

## Conclusions

In looking for indicators of traffic on I-94 westbound we found many useful results. First, there is more high volume traffic during the day than during the night. No surprise there, given that most people live their working lives and run most errands during the day. Second, traffic tends to increase throughout the day, no matter the day, but weekdays--business days--have strong peaks pre-work around 7:00 am and post-work around 4:00 pm. Third, we observed that there is not a large variation in traffic across months of the year, but the average traffic volume tends to be higher during the summer months (with the exception of July) compared to the winter months. This may be due to the fact that winter months entail snow, which people likely want to avoid. Fifth, weekday traffic is heavier than weekend traffic. People like to rest on the weekend--imagine that! Finally, we found that numerical weather indicators did not show a strong relationship to traffic, but some weather descriptions indicate higher or lower traffic trends: proximity rain showers slow traffic dramatically compared to other weather, whereas thunderstorms with heavy rain tend to keep people off of the road. It appears that people believe they can handle rain showers, but when there is thunder they prefer to stay inside.

Thus, if my business were to minimize the traffic faced on I-94, I would have my drivers deliver westbound at night, during the weekends, during the winter, and during heavy thunderstorms and fog. Of course, we might want to factor which of these same traffic indicators may increase risk of car accidents before we make any such decision.